In [2]:

import numpy as np

In [3]:
class OscillationGenerator(object):
    def __init__(self, w_f, a, b, q0, nf, use_deg):
        '''
        generate periodic oscillation from fourier series (Swevers, 1997)

        - w_f is the global pulsation (frequency is w_f / 2pi)
        - a and b are (arrays of) amplitudes of the sine/cosine
          functions for each joint
        - q0 is the joint angle offset (center of pulsation)
        - nf is the desired amount of coefficients for this fourier series
        '''
        self.w_f = float(w_f)
        self.a = a
        self.b = b
        self.use_deg = use_deg
        self.q0 = float(q0)
        if use_deg:
            self.q0 = np.deg2rad(self.q0)
        self.nf = nf

    def getAngle(self, t):
        #- t is the current time
        q = 0.0
        for l in range(1, self.nf+1):
            q += (self.a[l-1]/(self.w_f*l))*np.sin(self.w_f*l*t) - \
                 (self.b[l-1]/(self.w_f*l))*np.cos(self.w_f*l*t)
          
        q += self.nf*self.q0
        
        if self.use_deg:
            q = np.rad2deg(q)
        
        return q

    def getVelocity(self, t):
        dq = 0.0
        for l in range(1, self.nf+1):
            dq += self.a[l-1]*np.cos(self.w_f*l*t) + \
                  self.b[l-1]*np.sin(self.w_f*l*t)
        if self.use_deg:
            dq = np.rad2deg(dq)
        return dq

    def getAcceleration(self, t):
        ddq = 0.0
        for l in range(1, self.nf+1):
            ddq += -self.a[l-1]*self.w_f*l*np.sin(self.w_f*l*t) + \
                    self.b[l-1]*self.w_f*l*np.cos(self.w_f*l*t)
        if self.use_deg:
            ddq = np.rad2deg(ddq)
        return ddq
    
class Trajectory(object):
    ''' base trajectory class '''
    def getAngle(self, dof):
        raise NotImplementedError()

    def getVelocity(self, dof):
        raise NotImplementedError()

    def getAcceleration(self, dof):
        raise NotImplementedError()

    def getPeriodLength(self):
        raise NotImplementedError()

    def setTime(self, time):
        raise NotImplementedError()

    def wait_for_zero_vel(self, t_elapsed):
        raise NotImplementedError()

In [4]:
class PulsedTrajectory(Trajectory):
    ''' pulsating trajectory generator for one joint using fourier series from
        Swevers, Gansemann (1997). Gives values for one time instant (at the current
        internal time value)
    '''
    def __init__(self, dofs, use_deg=False):
        # type: (List, bool) -> None
        self.dofs = dofs
        self.oscillators = list()  # type: List[OscillationGenerator]
        self.use_deg = use_deg
        self.w_f_global = 1.0

   
    def initWithParams(self, a, b, q, nf, wf=None):
        ''' init with given params
            a - list of dof coefficients a
            b - list of dof coefficients b
            q - list of dof coefficients q_0
            nf - list of dof coefficients n_f
            (also see docstring of OscillationGenerator)
        '''

        if len(nf) != self.dofs or len(q) != self.dofs:
            raise Exception("Need DOFs many values for nf and q!")

        #for i in nf:
        #    if not ( len(a) == i and len(b) == i):
        #        raise Exception("Need nf many values in each parameter array value!")

        self.a = a
        self.b = b
        self.q = []
        self.nf = nf
        if wf:
            self.w_f_global = wf

        self.oscillators = list()
        for i in range(0, self.dofs):
            self.oscillators.append(OscillationGenerator(w_f = self.w_f_global, a = np.array(a[i]),
                                                         b = np.array(b[i]), q0 = q[i], nf = nf[i], use_deg = self.use_deg
                                                        ))
        return self

    def getAngle(self, dof):
        """ get angle at current time for joint dof """
        return self.oscillators[dof].getAngle(self.time)

    def getVelocity(self, dof):
        """ get velocity at current time for joint dof """
        return self.oscillators[dof].getVelocity(self.time)

    def getAcceleration(self, dof):
        """ get acceleration at current time for joint dof """
        return self.oscillators[dof].getAcceleration(self.time)

    def getPeriodLength(self):
        ''' get the period length of the oscillation in seconds '''
        return 2 * np.pi / self.w_f_global

    def setTime(self, time):
        '''set current time in seconds'''
        self.time = time

    def wait_for_zero_vel(self, t_elapsed):
        self.setTime(t_elapsed)
        if self.use_deg: thresh = 5.0
        else: thresh = np.deg2rad(5.0)
        return abs(self.getVelocity(0)) < thresh

In [5]:
from identification.data import Data
from identification.model import Model

def simulateTrajectory(trajectory, config):
    # type: (Dict, Trajectory, Model, np._ArrayLike) -> Tuple[Dict, Data]
    # generate data arrays for simulation and regressor building
    

   
    dofs=5
    
    trajectory_data = {}   # type: Dict[str, Union[List, np._ArrayLike]]
    trajectory_data['target_positions'] = []
    trajectory_data['target_velocities'] = []
    trajectory_data['target_accelerations'] = []
    trajectory_data['torques'] = []
    trajectory_data['times'] = []

    freq = 200
    for t in range(0, int(trajectory.getPeriodLength()*freq)):
        trajectory.setTime(t/freq)
        q = np.array([trajectory.getAngle(d) for d in range(dofs)])
        trajectory_data['target_positions'].append(q)
        #f.write("%s\n" % q)
        
        qdot = np.array([trajectory.getVelocity(d) for d in range(dofs)])
        trajectory_data['target_velocities'].append(qdot)

        qddot = np.array([trajectory.getAcceleration(d) for d in range(dofs)])
        trajectory_data['target_accelerations'].append(qddot)

        trajectory_data['times'].append(t/freq)
        trajectory_data['torques'].append(np.zeros(dofs))
   
    num_samples = len(trajectory_data['times'])
    
    #convert lists to numpy arrays
    trajectory_data['target_positions'] = np.array(trajectory_data['target_positions'])
    trajectory_data['positions'] = trajectory_data['target_positions']
    trajectory_data['target_velocities'] = np.array(trajectory_data['target_velocities'])
    trajectory_data['velocities'] = trajectory_data['target_velocities']
    trajectory_data['target_accelerations'] = np.array(trajectory_data['target_accelerations'])
    trajectory_data['accelerations'] = trajectory_data['target_accelerations']
    trajectory_data['torques'] = np.array(trajectory_data['torques'])
    trajectory_data['times'] = np.array(trajectory_data['times'])
    trajectory_data['measured_frequency'] = freq
    trajectory_data['base_velocity'] = np.zeros( (num_samples, 6) )
    trajectory_data['base_acceleration'] = np.zeros( (num_samples, 6) )

    trajectory_data['base_rpy'] = np.zeros( (num_samples, 3) )
    
    data = Data(config)
    data.init_from_data(trajectory_data)
    model.computeRegressors(data)
    
    return trajectory_data

In [6]:
dofs = 5
nf = 4

wf_min = 0.01
wf_max = 2.0
wf_init = 0.1

qmin = [-0.5]*dofs
qmax = [0.5]*dofs
qinit = [(0.5*(-0.436332)) + (0.5*(0.436332))]*dofs

amin = bmin = -1.5
amax = bmax = 1.5
ainit = [[0] * nf] * dofs
binit = [[0] * nf] * dofs
abnds = []
bbnds = []
qbnds = [None] * dofs
for i in range(0, dofs):
    qbnds[i] = (-0.5, 0.5)
    for j in range(0, nf):
        abnds.append((-1.5, 1.5))
        bbnds.append((-1.5, 1.5))
        ainit[i][j] = binit[i][j] = 0.3
wbnds = [(wf_min, wf_max)]

bnds = [wbnds, qbnds, abnds, bbnds]
bnds = [bnd for sublist in bnds for bnd in sublist]
print(bnds)

x0=[]
x0 = np.append([wf_init], qinit)
x0 = np.append(x0, np.ravel(ainit))
x0 = np.append(x0, np.ravel(binit))
x0 =[0.207237,0.000023247, -0.309111, -0.14981845, -0.2037325, -0.27274792,-0.5996635, 0.915887509, 0.3365387, -0.234871919,0.777777, -0.1020471, -0.035883, 0.9934340,1.0063, -0.8417213, -0.54149, 0.944568, 0.56095, 0.46266, 0.03673, -0.57127, 0.943396, -0.32849, 0.109994023, 0.70574,-0.381049, 0.832930, 1.0313, 0.8106638,-0.159298, 0.029010, 1.11428, 0.3471255, 0.23880, 1.01124, -0.01446305, 0.88860, 0.626896, -0.504415, -0.473716, -0.64263, -0.58491, -0.39494, 0.32540, -0.1499524]
print(len(x0),len(bnds))
def vecToParams(x):
        # convert vector of all solution variables to separate parameter variables
        nf = [4]*dofs
        wf = x[0]
        q = x[1:dofs+1]
        ab_len = dofs*nf[0]
        a = np.array(np.split(np.array(x[dofs+1:dofs+1+ab_len]), dofs))
        b = np.array(np.split(np.array(x[dofs+1+ab_len:dofs+1+ab_len*2]),dofs))
        return wf,q,a, b


[(0.01, 2.0), (-0.5, 0.5), (-0.5, 0.5), (-0.5, 0.5), (-0.5, 0.5), (-0.5, 0.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5)]
(46, 46)


In [7]:
from identification.model import Model
import iDynTree; iDynTree.init_helpers(); iDynTree.init_numpy_helpers()

import yaml


with open("/home/admin/rnd_test/FloBaRoID/configs/youbot_test.yaml", 'r') as stream:
    try:
        config = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

config['urdf'] = "/home/admin/rnd_test/FloBaRoID/model/youbot_arm.urdf"
model = Model(config, config['urdf'])

def objectiveFunc(x):
    wf, q, a, b = vecToParams(x)
    print(wf, q, a, b)
    nf = [4] * dofs
    trajectory = PulsedTrajectory(dofs, use_deg=0)
    trajectory.initWithParams(a, b, q, nf, wf)
    trajectory_data = simulateTrajectory(trajectory, config)
    
    return(np.linalg.cond(model.YBase))

def cont_1(x):
    wf, q, a, b = vecToParams(x)
    nf = [4] * dofs
    limits_low = [0.0, 0.0, -5.18363, 0.0, 0.0]
    trajectory = PulsedTrajectory(dofs, use_deg=0)
    trajectory.initWithParams(a, b, q, nf, wf)
    trajectory_data = simulateTrajectory(trajectory, config)
    
    for n in range(0, dofs):
        g_low = (limits_low[n]) - np.min(trajectory_data['positions'][:, n])
    
    return g_low

def cont_2(x):
    wf, q, a, b = vecToParams(x)
    nf = [4] * dofs
    limits_upper=[5.89921,2.70526,0,3.57792,5.7595] 
    trajectory = PulsedTrajectory(dofs, use_deg=0)
    trajectory.initWithParams(a, b, q, nf, wf)
    trajectory_data = simulateTrajectory(trajectory, config)
    
    for n in range(0, dofs):
        g_upper = np.max(trajectory_data['positions'][:, n]) - limits_upper[n]
    
    return g_upper

def cont_3(x):
    wf, q, a, b = vecToParams(x)
    nf = [4] * dofs
    limits_vel=[1.57079632679]*5 
    trajectory = PulsedTrajectory(dofs, use_deg=0)
    trajectory.initWithParams(a, b, q, nf, wf)
    trajectory_data = simulateTrajectory(trajectory, config)
    
    for n in range(0, dofs):
        g_vel = np.max(np.abs(trajectory_data['velocities'][:, n])) - limits_vel[n]
    
    return g_vel


con1 = {'type': 'ineq', 'fun': cont_1} 
con2 = {'type': 'ineq', 'fun': cont_2}
con3 = {'type': 'ineq', 'fun': cont_3}
cons = ([con1,con2,con3])    


    


loaded model /home/admin/rnd_test/FloBaRoID/model/youbot_arm.urdf
# DOFs: 5
Joints: ['arm_joint_1', 'arm_joint_2', 'arm_joint_3', 'arm_joint_4', 'arm_joint_5']
# regressor outputs: 5
# links: 6 (+ 0 fake)
{0: 'arm_link_0', 1: 'arm_link_1', 2: 'arm_link_2', 3: 'arm_link_3', 4: 'arm_link_4', 5: 'arm_link_5'}
# params: 60 (60 will be identified)
loaded random structural regressor from /home/admin/rnd_test/FloBaRoID/model/youbot_arm.urdf.regressor.npz


In [11]:
from identification.helpers import URDFHelpers

q = [2.1,1.8,-1.7,2.3,1.5] 
traversal = iDynTree.Traversal()
model = iDynTree.Model()
iDynTree.modelFromURDF('model/youbot_arm_only.urdf', model)
success = model.computeFullTreeTraversal(traversal)

rotation = iDynTree.Rotation(1.0, 0.0, 0.0,
                             0.0, 1.0, 0.0,
                             0.0, 0.0, 1.0)
position = iDynTree.Position(0.0, 0.0, 0.0)
worldHbase = iDynTree.Transform(rotation, position)
jointPosition = iDynTree.VectorDynSize(5)

for i in range(len(q)):
    jointPosition[i] = q[i]

linkPositions = iDynTree.LinkPositions(model) # 5, 7???

iDynTree.ForwardPositionKinematics(model, traversal, worldHbase,
jointPosition, linkPositions)

pose_str = linkPositions.toString(model)
pose_link5 = linkPositions(5)
print(pose_link5.getPosition())

0.288373 0.208188 0.352922


In [9]:
print(pose_link5.getPosition().getVal(0))

0.288372965895


In [80]:
#pose_str.find("_5:")
str_cut = pose_str[576:684]
print((str_cut))
pose_endeff = map(float , str_cut[0:28].split(' '))
#l.append(map(float , str_cut[29:54].split(' ')))
#l.append(map(float , str_cut[55:77].split(' ')))
#l.append(map(float , str_cut[78:109].split(' ')))



-0.272663 -0.615975 0.739073
0.718886 0.380097 0.582005
-0.63942 0.69 0.339177
 0.288373 0.208188 0.352922




[-0.272663, -0.615975, 0.739073]